In [ ]:
# %%file main.py
"""
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!pip install torchvision
!pip install torch==1.4.0
!pip install torchaudio==0.4.0
%matplotlib inline
!python pytorch-xla-env-setup.py --version $VERSION
"""


# !pip install torch_intermediate_layer_getter
# from torch_intermediate_layer_getter import IntermediateLayerGetter as MidGetter


# !pip install line_profiler
# %load_ext line_profiler

#standard
import os
import time
import matplotlib.pyplot as plt
from IPython import display
from typing import List, Tuple
import numpy as np
from google.colab import drive
import math
import sys

from mpl_toolkits.mplot3d.axes3d import Axes3D


#torch
import torch
import torch.jit as jit
from torch import Tensor
import torchvision


import time
import tensorflow as tf
from IPython.display import clear_output

import pickle

#import torch_xla
#import torch_xla.core.xla_model as xm

# !pip install py-heat-magic
# %load_ext heat



# !pip install memory_profiler
# %load_ext memory_profiler

DEVICE = "cpu"


class Flatten(torch.nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def loss_penalized(output, target, x, input):
    (x_OUTDOORTEMPERATURE_input,
        x_RADIATION_input,
        x_SPACEHEATER_input,
        x_VENTILATION_input) = input

    

    (x_OUTDOORTEMPERATURE_output, x_RADIATION_output, x_SPACEHEATER_output, x_VENTILATION_output) = x


    tol = 1e-8

    # DELTA_x_OUTDOORTEMPERATURE_output = torch.zeros(x_OUTDOORTEMPERATURE_output.shape).cuda()
    # DELTA_x_OUTDOORTEMPERATURE_output[:,1:] = x_OUTDOORTEMPERATURE_output[:,1:]-x_OUTDOORTEMPERATURE_output[:,:-1]
    # DELTA_x_OUTDOORTEMPERATURE_input_0 = torch.zeros(x_OUTDOORTEMPERATURE_output.shape).cuda()
    # DELTA_x_OUTDOORTEMPERATURE_input_0[:,1:,0] = x_OUTDOORTEMPERATURE_input[:,1:,0]-x_OUTDOORTEMPERATURE_input[:,:-1,0]
    # DELTA_x_OUTDOORTEMPERATURE_input_1 = torch.zeros(x_OUTDOORTEMPERATURE_output.shape).cuda()
    # DELTA_x_OUTDOORTEMPERATURE_input_1[:,1:,0] = x_OUTDOORTEMPERATURE_input[:,1:,1]-x_OUTDOORTEMPERATURE_input[:,:-1,1]
    # bool_arr_grad_neg = torch.logical_and(DELTA_x_OUTDOORTEMPERATURE_input_0 > -tol, DELTA_x_OUTDOORTEMPERATURE_input_1 < tol)
    # bool_arr_grad_pos = torch.logical_and(DELTA_x_OUTDOORTEMPERATURE_input_0 < tol, DELTA_x_OUTDOORTEMPERATURE_input_1 > -tol)
    # loss_OUTDOORTEMPERATURE = torch.zeros(x_OUTDOORTEMPERATURE_output.shape).cuda()
    # loss_OUTDOORTEMPERATURE[bool_arr_grad_neg] = torch.relu(DELTA_x_OUTDOORTEMPERATURE_output[bool_arr_grad_neg])
    # loss_OUTDOORTEMPERATURE[bool_arr_grad_pos] = torch.relu(-DELTA_x_OUTDOORTEMPERATURE_output[bool_arr_grad_pos])


    grad_OUTDOORTEMPERATURE = torch.autograd.grad(
            x_OUTDOORTEMPERATURE_output, x_OUTDOORTEMPERATURE_input,
            grad_outputs=torch.ones_like(x_OUTDOORTEMPERATURE_output),
            retain_graph=True,
            create_graph=True
        )[0]
    loss_OUTDOORTEMPERATURE_0 = torch.relu(grad_OUTDOORTEMPERATURE[:,:,0].unsqueeze(2))
    loss_OUTDOORTEMPERATURE_1 = torch.relu(-grad_OUTDOORTEMPERATURE[:,:,1].unsqueeze(2))


    # loss_RADIATION = torch.zeros(x_RADIATION_output.shape).to(DEVICE)
    # bool_arr = x_RADIATION_input[:,:,0] < tol
    # loss_RADIATION[bool_arr] = torch.abs(x_RADIATION_output[bool_arr])
    # grad_RADIATION = torch.autograd.grad(
    #         x_RADIATION_output, x_RADIATION_input,
    #         grad_outputs=torch.ones_like(x_RADIATION_output),
    #         retain_graph=True,
    #         create_graph=True
    #     )[0]
    # loss_RADIATION_0 = torch.relu(grad_RADIATION[:,:,0].unsqueeze(2))

    DELTA_x_SPACEHEATER_output = torch.zeros(x_SPACEHEATER_output.shape).to(DEVICE)
    DELTA_x_SPACEHEATER_output[:,1:] = x_SPACEHEATER_output[:,1:]-x_SPACEHEATER_output[:,:-1]
    bool_arr_grad = torch.logical_and(x_SPACEHEATER_input[:,:,1] < tol, x_SPACEHEATER_output[:,:,0] > tol)
    # bool_arr_constant = torch.logical_and(torch.abs(DELTA_x_SPACEHEATER_output[:,:,0]) < tol, x_SPACEHEATER_input[:,:,1] < tol)
    loss_SPACEHEATER = torch.zeros(x_SPACEHEATER_output.shape).to(DEVICE)
    loss_SPACEHEATER[bool_arr_grad] = torch.relu(DELTA_x_SPACEHEATER_output[bool_arr_grad])
    # loss_SPACEHEATER[bool_arr_constant] = torch.relu(x_SPACEHEATER_output[bool_arr_constant])

    grad_SPACEHEATER = torch.autograd.grad(
            x_SPACEHEATER_output, x_SPACEHEATER_input,
            grad_outputs=torch.ones_like(x_SPACEHEATER_output),
            retain_graph=True,
            create_graph=True
        )[0]
    loss_SPACEHEATER_0 = torch.relu(grad_SPACEHEATER[:,:,0].unsqueeze(2))
    loss_SPACEHEATER_1 = torch.relu(-grad_SPACEHEATER[:,:,1].unsqueeze(2))
    loss_SPACEHEATER_2 = torch.relu(-grad_SPACEHEATER[:,:,2].unsqueeze(2))



    loss_VENTILATION = torch.zeros(x_VENTILATION_output.shape).to(DEVICE)
    bool_arr = x_VENTILATION_input[:,:,1] < tol
    loss_VENTILATION[bool_arr] = torch.abs(x_VENTILATION_output[bool_arr])

    grad_VENTILATION = torch.autograd.grad(
            x_VENTILATION_output, x_VENTILATION_input,
            grad_outputs=torch.ones_like(x_VENTILATION_output),
            retain_graph=True,
            create_graph=True
        )[0]
    loss_VENTILATION_0 = torch.relu(grad_VENTILATION[:,:,0].unsqueeze(2))
    loss_VENTILATION_1 = torch.relu(-grad_VENTILATION[:,:,1].unsqueeze(2))
    loss_VENTILATION_2 = torch.relu(-grad_VENTILATION[:,:,2].unsqueeze(2))

    


    K = 1
    loss = torch.mean(
        (output - target)**2 + 
        K*torch.relu(-x_SPACEHEATER_output) + 
        K*torch.relu(-x_RADIATION_output) + 
        # K*loss_OUTDOORTEMPERATURE + 
        K*loss_OUTDOORTEMPERATURE_0 + 
        K*loss_OUTDOORTEMPERATURE_1 + 
        # K*loss_RADIATION + 
        # K*loss_RADIATION_0 + 
        K*loss_SPACEHEATER + 
        K*loss_SPACEHEATER_0 + 
        K*loss_SPACEHEATER_1 +
        K*loss_SPACEHEATER_2 +
        # K*loss_VENTILATION +
        K*loss_VENTILATION_0 + 
        K*loss_VENTILATION_1 +
        K*loss_VENTILATION_2)
        
    return loss

# class LSTM(jit.ScriptModule):
class LSTM(torch.nn.Module):

    def __init__(self, 
                 n_input=None, 
                 n_lstm_hidden=None, 
                 n_lstm_layers=None, 
                 n_output=None, 
                 dropout=None,
                 scaling_value_dict=None):

        self.kwargs = {"n_input": n_input,
                        "n_lstm_hidden": n_lstm_hidden,
                        "n_lstm_layers": n_lstm_layers,
                        "n_output": n_output,
                        "dropout": dropout,
                        "scaling_value_dict": scaling_value_dict}
        
        self.n_input = n_input
    
        (self.n_lstm_hidden_OUTDOORTEMPERATURE,
        self.n_lstm_hidden_RADIATION,
        self.n_lstm_hidden_SPACEHEATER,
        self.n_lstm_hidden_VENTILATION) = n_lstm_hidden
        

        (self.n_lstm_layers_OUTDOORTEMPERATURE,
        self.n_lstm_layers_RADIATION,
        self.n_lstm_layers_SPACEHEATER,
        self.n_lstm_layers_VENTILATION) = n_lstm_layers

        self.n_output = n_output

        self.dropout = dropout

        super(LSTM, self).__init__()

        # self.lstm_hidden1_OUTDOORTEMPERATURE = torch.nn.LSTM(self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_layers_OUTDOORTEMPERATURE, batch_first=True, dropout=self.dropout, bias=False)
        # self.lstm_hidden2_OUTDOORTEMPERATURE = torch.nn.LSTM(self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_layers_OUTDOORTEMPERATURE, batch_first=True, dropout=self.dropout, bias=False)
        # self.lstm_hidden3_OUTDOORTEMPERATURE = torch.nn.LSTM(self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_layers_OUTDOORTEMPERATURE, batch_first=True, dropout=self.dropout, bias=False)
        self.lstm_input_OUTDOORTEMPERATURE = torch.nn.LSTM(2, self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_lstm_layers_OUTDOORTEMPERATURE, batch_first=True, dropout=self.dropout, bias=False)
        self.lstm_output_OUTDOORTEMPERATURE = torch.nn.LSTM(self.n_lstm_hidden_OUTDOORTEMPERATURE, self.n_output, 1, batch_first=True, bias=False)


        # self.T_w_in__cp = torch.nn.Parameter(torch.randn(1))
        # self.m_w_max = torch.nn.Parameter(torch.randn(1))
        # self.UA = torch.nn.Parameter(torch.randn(1))

        # self.lstm_hidden1_RADIATION = torch.nn.LSTM(self.n_lstm_hidden_RADIATION, self.n_lstm_hidden_RADIATION, self.n_lstm_layers_RADIATION, batch_first=True, dropout=self.dropout, bias=False)
        # self.lstm_hidden2_RADIATION = torch.nn.LSTM(self.n_lstm_hidden_RADIATION, self.n_lstm_hidden_RADIATION, self.n_lstm_layers_RADIATION, batch_first=True, dropout=self.dropout, bias=False)
        # self.lstm_hidden3_RADIATION = torch.nn.LSTM(self.n_lstm_hidden_RADIATION, self.n_lstm_hidden_RADIATION, self.n_lstm_layers_RADIATION, batch_first=True, dropout=self.dropout, bias=False)
        self.lstm_input_RADIATION = torch.nn.LSTM(1, self.n_lstm_hidden_RADIATION, self.n_lstm_layers_RADIATION, batch_first=True, dropout=self.dropout, bias=False)
        self.lstm_output_RADIATION = torch.nn.LSTM(self.n_lstm_hidden_RADIATION, self.n_output, 1, batch_first=True, bias=False)

        self.lstm_input_SPACEHEATER = torch.nn.LSTM(3, self.n_lstm_hidden_SPACEHEATER, self.n_lstm_layers_SPACEHEATER, batch_first=True, dropout=self.dropout, bias=False)
        self.lstm_output_SPACEHEATER = torch.nn.LSTM(self.n_lstm_hidden_SPACEHEATER, self.n_output, 1, batch_first=True, bias=False)

        self.lstm_input_VENTILATION = torch.nn.LSTM(3, self.n_lstm_hidden_VENTILATION, self.n_lstm_layers_VENTILATION, batch_first=True, dropout=self.dropout, bias=False)
        self.lstm_output_VENTILATION = torch.nn.LSTM(self.n_lstm_hidden_VENTILATION, self.n_output, 1, batch_first=True, bias=False)

        self.dropout = torch.nn.Dropout(p=self.dropout, inplace=False)


        # self.linear_OUTDOORTEMPERATURE = torch.nn.Linear(self.n_lstm_hidden_OUTDOORTEMPERATURE,1)
        # self.linear_RADIATION = torch.nn.Linear(self.n_lstm_hidden_RADIATION,1)
        # self.linear_SPACEHEATER = torch.nn.Linear(self.n_lstm_hidden_SPACEHEATER,1)
        # self.linear_VENTILATION = torch.nn.Linear(self.n_lstm_hidden_VENTILATION,1)

        self.linear_u = torch.nn.Linear(1,1, bias=False)
        self.linear_T_o_hid = torch.nn.Linear(1,self.n_lstm_hidden_VENTILATION)
        self.linear_T_o_out = torch.nn.Linear(self.n_lstm_hidden_VENTILATION,1)
        self.linear_T_z = torch.nn.Linear(1,1)

        self.tanh = torch.nn.Tanh()
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()



        
        
    # @jit.script_method
    def forward(self, 
                input: Tuple[Tensor, Tensor, Tensor, Tensor], 
                hidden_state: Tuple[Tuple[Tensor, Tensor], 
                                    Tuple[Tensor, Tensor], 
                                    Tuple[Tensor, Tensor], 
                                    Tuple[Tensor, Tensor],
                                    Tuple[Tensor, Tensor], 
                                    Tuple[Tensor, Tensor], 
                                    Tuple[Tensor, Tensor], 
                                    Tuple[Tensor, Tensor]]):


        (x_OUTDOORTEMPERATURE,
        x_RADIATION,
        x_SPACEHEATER,
        x_VENTILATION) = input

        

        (hidden_state_input_OUTDOORTEMPERATURE,
        hidden_state_output_OUTDOORTEMPERATURE,
        hidden_state_input_RADIATION,
        hidden_state_output_RADIATION,
        hidden_state_input_SPACEHEATER,
        hidden_state_output_SPACEHEATER,
        hidden_state_input_VENTILATION,
        hidden_state_output_VENTILATION) = hidden_state

        # h_0_hidden1_OUTDOORTEMPERATURE = torch.zeros(hidden_state_input_OUTDOORTEMPERATURE[0].shape).cuda()
        # c_0_hidden1_layer_OUTDOORTEMPERATURE = torch.zeros(hidden_state_input_OUTDOORTEMPERATURE[1].shape).cuda()
        # hidden_state_hidden1_OUTDOORTEMPERATURE = (h_0_hidden1_OUTDOORTEMPERATURE,c_0_hidden1_layer_OUTDOORTEMPERATURE)
        # h_0_hidden2_OUTDOORTEMPERATURE = torch.zeros(hidden_state_input_OUTDOORTEMPERATURE[0].shape).cuda()
        # c_0_hidden2_layer_OUTDOORTEMPERATURE = torch.zeros(hidden_state_input_OUTDOORTEMPERATURE[1].shape).cuda()
        # hidden_state_hidden2_OUTDOORTEMPERATURE = (h_0_hidden2_OUTDOORTEMPERATURE,c_0_hidden2_layer_OUTDOORTEMPERATURE)

        # h_0_hidden1_RADIATION = torch.zeros(hidden_state_input_RADIATION[0].shape).cuda()
        # c_0_hidden1_layer_RADIATION = torch.zeros(hidden_state_input_RADIATION[1].shape).cuda()
        # hidden_state_hidden1_RADIATION = (h_0_hidden1_RADIATION,c_0_hidden1_layer_RADIATION)
        # h_0_hidden2_RADIATION = torch.zeros(hidden_state_input_RADIATION[0].shape).cuda()
        # c_0_hidden2_layer_RADIATION = torch.zeros(hidden_state_input_RADIATION[1].shape).cuda()
        # hidden_state_hidden2_RADIATION = (h_0_hidden2_RADIATION,c_0_hidden2_layer_RADIATION)


        x_OUTDOORTEMPERATURE,hidden_state_input_OUTDOORTEMPERATURE = self.lstm_input_OUTDOORTEMPERATURE(x_OUTDOORTEMPERATURE,hidden_state_input_OUTDOORTEMPERATURE)
        # x_OUTDOORTEMPERATURE = x_OUTDOORTEMPERATURE + x_OUTDOORTEMPERATURE_temp
        # x_OUTDOORTEMPERATURE_temp,hidden_state_input_OUTDOORTEMPERATURE = self.lstm_hidden1_OUTDOORTEMPERATURE(x_OUTDOORTEMPERATURE,hidden_state_hidden1_OUTDOORTEMPERATURE)
        # x_OUTDOORTEMPERATURE = x_OUTDOORTEMPERATURE + x_OUTDOORTEMPERATURE_temp
        # x_OUTDOORTEMPERATURE_temp,hidden_state_input_OUTDOORTEMPERATURE = self.lstm_hidden2_OUTDOORTEMPERATURE(x_OUTDOORTEMPERATURE,hidden_state_hidden2_OUTDOORTEMPERATURE)
        # x_OUTDOORTEMPERATURE = x_OUTDOORTEMPERATURE + x_OUTDOORTEMPERATURE_temp
        # x_OUTDOORTEMPERATURE,hidden_state_input_OUTDOORTEMPERATURE = self.lstm_input_OUTDOORTEMPERATURE(x_OUTDOORTEMPERATURE,hidden_state_input_OUTDOORTEMPERATURE)
        # x_OUTDOORTEMPERATURE = self.dropout(x_OUTDOORTEMPERATURE)
        # x_OUTDOORTEMPERATURE = x_OUTDOORTEMPERATURE + x_OUTDOORTEMPERATURE_
        x_OUTDOORTEMPERATURE,hidden_state_output_OUTDOORTEMPERATURE = self.lstm_output_OUTDOORTEMPERATURE(x_OUTDOORTEMPERATURE,hidden_state_output_OUTDOORTEMPERATURE)

        x_RADIATION,hidden_state_input_RADIATION = self.lstm_input_RADIATION(x_RADIATION,hidden_state_input_RADIATION)
        # x_RADIATION = x_RADIATION + x_RADIATION_temp
        # x_RADIATION_temp,hidden_state_input_RADIATION = self.lstm_hidden1_RADIATION(x_RADIATION,hidden_state_hidden1_RADIATION)
        # x_RADIATION = x_RADIATION + x_RADIATION_temp
        # x_RADIATION_temp,hidden_state_input_RADIATION = self.lstm_hidden2_RADIATION(x_RADIATION,hidden_state_hidden2_RADIATION)
        # x_RADIATION = x_RADIATION + x_RADIATION_temp
        x_RADIATION,hidden_state_output_RADIATION = self.lstm_output_RADIATION(x_RADIATION,hidden_state_output_RADIATION)

        x_SPACEHEATER,hidden_state_input_SPACEHEATER = self.lstm_input_SPACEHEATER(x_SPACEHEATER,hidden_state_input_SPACEHEATER)
        # x_SPACEHEATER = self.dropout(x_SPACEHEATER)
        # x_SPACEHEATER = x_SPACEHEATER + x_SPACEHEATER_
        x_SPACEHEATER,hidden_state_output_SPACEHEATER = self.lstm_output_SPACEHEATER(x_SPACEHEATER,hidden_state_output_SPACEHEATER)

        # x_damper = torch.unsqueeze(x_VENTILATION[:,:,1],2)
        x_VENTILATION,hidden_state_input_VENTILATION = self.lstm_input_VENTILATION(x_VENTILATION,hidden_state_input_VENTILATION)
        # # x_VENTILATION = self.dropout(x_VENTILATION)
        # # x_VENTILATION = x_VENTILATION + x_VENTILATION_
        x_VENTILATION,hidden_state_output_VENTILATION = self.lstm_output_VENTILATION(x_VENTILATION,hidden_state_output_VENTILATION)
        # x_VENTILATION = x_VENTILATION*x_damper


        # d1,d2,d3 = x_VENTILATION.size()
        # x_VENTILATION = x_VENTILATION.reshape(d1*d2,d3)
        # x_indoor = torch.unsqueeze(x_VENTILATION[:,0],1)
        # x_damper = torch.unsqueeze(x_VENTILATION[:,1],1)
        # x_supply_air_temperature = torch.unsqueeze(x_VENTILATION[:,2],1)

        # x_VENTILATION = (x_supply_air_temperature-self.linear_T_z(x_indoor))*self.linear_u(x_damper)
        # x_VENTILATION = x_VENTILATION.reshape(d1,d2,1)

        x = (x_OUTDOORTEMPERATURE, x_RADIATION, x_SPACEHEATER, x_VENTILATION)
        y = x_OUTDOORTEMPERATURE + x_RADIATION + x_SPACEHEATER + x_VENTILATION







        hidden_state = (hidden_state_input_OUTDOORTEMPERATURE,
                        hidden_state_output_OUTDOORTEMPERATURE,
                        hidden_state_input_RADIATION,
                        hidden_state_output_RADIATION,
                        hidden_state_input_SPACEHEATER,
                        hidden_state_output_SPACEHEATER,
                        hidden_state_input_VENTILATION,
                        hidden_state_output_VENTILATION)

        return y,hidden_state,x







def min_max_norm(y,y_min,y_max,low,high):
    y = (y-y_min)/(y_max-y_min)*(high-low) + low
    return y

def rescale(y,y_min,y_max,low,high):
    y = (y-low)/(high-low)*(y_max-y_min) + y_min
    return y

def sigmoid_inverse(x):
    out = np.log(x/(1-x))
    return out






class Trainer:
    def __init__(self, space_name, data_paths, load=True, plot=False):

        self.space_name = space_name

        (self.training_data_folder_name,
        self.drive_zipped_training_data_path,
        self.drive_unzipped_training_data_path,
        self.vm_drive_unzip_data_path,
        self.vm_drive_training_data_path) = data_paths


        np.random.seed(0)

        self.saved_serialized_networks_path = "/content/drive/My Drive/Google collab/b3d_serialized_networks_new/"
        self.saved_networks_path = "/content/drive/My Drive/Google collab/b3d_saved_networks_new/"

        os.chdir(self.saved_networks_path)

        # self.T_min = -7.91
        # self.T_max = 39.29999923706055
        
        self.dT_min = -1
        self.dT_max = -self.dT_min
        
        self.best_loss_diff_max = 5000
        self.max_it_stop = 1000000
        momentum = 0.9 
        LR = '1e-2' #1e-1
        LR_s = '1e-2' #1e-1
        self.update_factor = 1e-1
        self.n_step_diff = 10000000000 #768000 #30 Epochs
        self.n_step_update = self.n_step_diff
        self.n_step_max = int(self.n_step_diff+self.n_step_diff/4+self.n_step_diff/16)
        self.learning_rate = float(LR_s)
        self.n_batch = 2**6
        self.n_batch_s = 2**6
        # self.file_batch = 64
        self.file_batch = 2**9 #2048
        self.n_output = 1
        self.n_input = self.get_input_size() ################################################
        self.n_lstm_hidden = (5,5,5,5) #20 #(2,2,1,3) good
        self.n_lstm_layers = (3,3,3,3)
        self.dropout = 0.


        self.load_min_max_scale_values()
        self.model = LSTM(self.n_input, self.n_lstm_hidden, self.n_lstm_layers, self.n_output, self.dropout, self.scaling_value_dict).to(DEVICE)
        self.model.train()

        

        # self.optimizer = torch.optim.NAdam(self.model.parameters(), lr=self.learning_rate)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        # self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate,momentum=momentum)
        self.loss_fn = loss_penalized#torch.nn.MSELoss()

        # model_type_load = "b3d_B%d_LR%s" % (self.n_batch,LR)
        # model_type_save = "b3d_B%d_LR%s" % (self.n_batch,LR_s)
        # model_type_load = "T_diff_b3d_B%d_LR%s" % (self.n_batch,LR)
        # model_type_save = "T_diff_b3d_B%d_LR%s" % (self.n_batch,LR_s)
        # model_type_load = "T_diff_LSTM_b3d_B%d_LR%s" % (self.n_batch,LR)
        # model_type_save = "T_diff_LSTM_b3d_B%d_LR%s" % (self.n_batch_s,LR_s)
        model_type_load = self.space_name + "_T_diff_LSTM_b3d_B%d_LR%s" % (self.n_batch,LR)
        model_type_save = self.space_name + "_T_diff_LSTM_b3d_B%d_LR%s" % (self.n_batch_s,LR_s)

        self.network_filename_load = self.space_name + "_Network"
        self.optimizer_filename_load = self.space_name + "_Optimizer"

        self.network_filename_save = self.space_name + "_Network"
        self.optimizer_filename_save = self.space_name + "_Optimizer"

        self.step_train_filename_load = self.space_name + "_step_train"
        self.prec_train_filename_load = self.space_name + "_prec_train"

        self.step_train_filename_save = self.space_name + "_step_train"
        self.prec_train_filename_save = self.space_name + "_prec_train"

        self.step_test_filename_load = self.space_name + "_step_test"
        self.prec_test_filename_load = self.space_name + "_prec_test"

        self.step_test_filename_save = self.space_name + "_step_test"
        self.prec_test_filename_save = self.space_name + "_prec_test"

        self.running_validation_loss_filename_load = self.space_name + "_running_validation_loss"
        self.running_validation_loss_filename_save = self.space_name + "_running_validation_loss"

        self.running_validation_loss_model_name_filename_load = self.space_name + "_running_validation_loss_model_name"
        self.running_validation_loss_model_name_filename_save = self.space_name + "_running_validation_loss_model_name"

        if load==True:
            os.chdir(self.saved_networks_path)
            self.model.load_state_dict(torch.load(self.network_filename_load,map_location=torch.device(DEVICE)))
            self.optimizer.load_state_dict(torch.load(self.optimizer_filename_load,map_location=torch.device(DEVICE)))

            self.step_train = np.load(self.step_train_filename_load + ".npy",allow_pickle=True).tolist()
            self.prec_train = np.load(self.prec_train_filename_load + ".npy",allow_pickle=True).tolist()

            self.step_test = np.load(self.step_test_filename_load + ".npy",allow_pickle=True).tolist()
            self.prec_test = np.load(self.prec_test_filename_load + ".npy",allow_pickle=True).tolist()

            os.chdir(self.saved_serialized_networks_path)

            self.running_validation_loss = np.load(self.running_validation_loss_filename_load + ".npy",allow_pickle=True).tolist()
            self.running_validation_loss_model_name = np.load(self.running_validation_loss_model_name_filename_load + ".npy",allow_pickle=True).tolist()

            os.chdir(self.saved_networks_path)

            self.n_step = self.step_train[-1]
        else:
            load_pretrained_model = False   
            if load_pretrained_model:
                print(os.getcwd())
                print(os.listdir())
                self.model.load_state_dict(torch.load(self.network_filename_load + ".pt",map_location=torch.device(DEVICE)))
            
            self.step_train = []
            self.prec_train = [] 
            self.step_test = []
            self.prec_test = [] 
            self.running_validation_loss = []
            self.running_validation_loss_model_name = []
            self.n_step = 0
        pytorch_total_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print("TOTAL NUMBER OF PARAMETERS IN MODEL: " + str(pytorch_total_params))

        self.n_step_start = self.n_step



        self.batch_idx_train = 0
        self.batch_idx_test = 0
        self.batch_train_counter = 0

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.learning_rate

        self.do_test = True
        self.extract_model = True

        self.n_test = 2**4
        self.plot = plot
        

        self.n_save_traced_module = self.n_test


        




        h_0_input_layer_OUTDOORTEMPERATURE = torch.zeros((self.n_lstm_layers[0],self.n_batch_s,self.n_lstm_hidden[0])).to(DEVICE)
        c_0_input_layer_OUTDOORTEMPERATURE = torch.zeros((self.n_lstm_layers[0],self.n_batch_s,self.n_lstm_hidden[0])).to(DEVICE)
        h_0_output_layer_OUTDOORTEMPERATURE = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        c_0_output_layer_OUTDOORTEMPERATURE = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        hidden_state_input_OUTDOORTEMPERATURE = (h_0_input_layer_OUTDOORTEMPERATURE,c_0_input_layer_OUTDOORTEMPERATURE)
        hidden_state_output_OUTDOORTEMPERATURE = (h_0_output_layer_OUTDOORTEMPERATURE,c_0_output_layer_OUTDOORTEMPERATURE)

        h_0_input_layer_RADIATION = torch.zeros((self.n_lstm_layers[1],self.n_batch_s,self.n_lstm_hidden[1])).to(DEVICE)
        c_0_input_layer_RADIATION = torch.zeros((self.n_lstm_layers[1],self.n_batch_s,self.n_lstm_hidden[1])).to(DEVICE)
        h_0_output_layer_RADIATION = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        c_0_output_layer_RADIATION = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        hidden_state_input_RADIATION = (h_0_input_layer_RADIATION,c_0_input_layer_RADIATION)
        hidden_state_output_RADIATION = (h_0_output_layer_RADIATION,c_0_output_layer_RADIATION)

        h_0_input_layer_SPACEHEATER = torch.zeros((self.n_lstm_layers[2],self.n_batch_s,self.n_lstm_hidden[2])).to(DEVICE)
        c_0_input_layer_SPACEHEATER = torch.zeros((self.n_lstm_layers[2],self.n_batch_s,self.n_lstm_hidden[2])).to(DEVICE)
        h_0_output_layer_SPACEHEATER = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        c_0_output_layer_SPACEHEATER = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        hidden_state_input_SPACEHEATER = (h_0_input_layer_SPACEHEATER,c_0_input_layer_SPACEHEATER)
        hidden_state_output_SPACEHEATER = (h_0_output_layer_SPACEHEATER,c_0_output_layer_SPACEHEATER)

        h_0_input_layer_VENTILATION = torch.zeros((self.n_lstm_layers[3],self.n_batch_s,self.n_lstm_hidden[3])).to(DEVICE)
        c_0_input_layer_VENTILATION = torch.zeros((self.n_lstm_layers[3],self.n_batch_s,self.n_lstm_hidden[3])).to(DEVICE)
        h_0_output_layer_VENTILATION = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        c_0_output_layer_VENTILATION = torch.zeros((1,self.n_batch_s,self.n_output)).to(DEVICE)
        hidden_state_input_VENTILATION = (h_0_input_layer_VENTILATION,c_0_input_layer_VENTILATION)
        hidden_state_output_VENTILATION = (h_0_output_layer_VENTILATION,c_0_output_layer_VENTILATION)



        self.hidden_state = (hidden_state_input_OUTDOORTEMPERATURE,
                            hidden_state_output_OUTDOORTEMPERATURE,
                            hidden_state_input_RADIATION,
                            hidden_state_output_RADIATION,
                            hidden_state_input_SPACEHEATER,
                            hidden_state_output_SPACEHEATER,
                            hidden_state_input_VENTILATION,
                            hidden_state_output_VENTILATION)
    


    def load_dataset_into_RAM(self):
        drive.mount('/content/drive')
        os.chdir(self.training_data_path) 
        self.cnn_input_list = []
        self.flat_input_list = []
        self.h_0_input_list = []
        self.output_list = []


        self.it_list = [0,1] #########################################
        for i in self.it_list:
            print(i)
            name_idx_rand = self.file_batch*i + self.file_batch
            filename = 'Batch_%d.npz' % (name_idx_rand)

        
            loaded = np.load(filename)
            cnn_input = torch.Tensor(loaded[loaded.files[0]])
            flat_input = torch.Tensor(loaded[loaded.files[1]])
            h_0_input = torch.Tensor(loaded[loaded.files[2]])
            output = torch.Tensor(loaded[loaded.files[3]])
            del loaded.f
            loaded.close()

            print("----")
            print(cnn_input.element_size())
            print(cnn_input.nelement())
            print(cnn_input.element_size() * cnn_input.nelement())

            flat_input_temp = torch.zeros((flat_input.shape[0],flat_input.shape[1],flat_input.shape[2]-1))
            flat_input_temp[:,:,0:2] = flat_input[:,:,0:2]
            flat_input_temp[:,:,2:] = flat_input[:,:,3:]

            flat_input = flat_input_temp

            self.cnn_input_list.append(cnn_input)
            self.flat_input_list.append(flat_input)
            self.h_0_input_list.append(h_0_input)
            self.output_list.append(output)

            
    def get_input_size(self):
        os.chdir(self.vm_drive_training_data_path)
        filename = self.space_name + "_training_batch_%d.npz" % (self.file_batch)
        loaded = np.load(filename)
        flat_input = torch.Tensor(loaded[loaded.files[0]])
        os.chdir(self.saved_networks_path)
        n_input = flat_input.shape[2]
        
        print("INPUT")
        print(n_input)

        return n_input



    def load_batch_into_GPU(self):
        os.chdir(self.vm_drive_training_data_path)
        # os.chdir(self.drive_unzipped_training_data_path)
        
        
        if self.batch_idx_train == self.max_it_train:
            np.random.shuffle(self.it_list_train)
            self.batch_idx_train = 0
        
        if self.batch_idx_test==self.max_it_test:
            np.random.shuffle(self.it_list_test)
            self.batch_idx_test = 0


        # if self.batch_train_counter % self.test_it == 0:
        #     self.do_test = True
        #     self.first_test = False


        if self.do_test==False:
            i = self.it_list_train[self.batch_idx_train]
            self.data_type = "training"
            
        else:
            i = self.it_list_test[self.batch_idx_test]
            self.data_type = "validation"
        
        name_idx_rand = self.file_batch*i + self.file_batch
        filename = self.space_name + "_" + self.data_type + "_batch_%d.npz" % (name_idx_rand)

        loaded = np.load(filename)


        flat_input = torch.Tensor(loaded[loaded.files[0]])
        output = torch.Tensor(loaded[loaded.files[1]])  

        self.flat_input = flat_input.to(DEVICE)
        self.output = output.to(DEVICE)

        temp = self.flat_input
        # self.flat_input = self.flat_input[:,1:]
        # self.flat_input[:,:,0] = temp[:,:-1,0] #Indoor temperature

        self.flat_input = self.flat_input[:,:-1]





        # self.h_0_input = self.h_0_input[:,:-1]
        self.output_dT = self.output[:,1:]-self.output[:,:-1]
        # self.output = self.output




        # self.output_dT = min_max_norm(self.output_dT,self.dT_min,self.dT_max,-1,1)



    def load_min_max_scale_values(self):
        os.chdir(self.vm_drive_training_data_path)
        filename = self.space_name + "_scaling_value_dict" + ".pickle"
        filehandler = open(filename, 'rb')
        self.scaling_value_dict = pickle.load(filehandler)

    def scan_directory(self):
        os.chdir(self.vm_drive_training_data_path)
        directory = os.fsencode(self.vm_drive_training_data_path)
        file_counter_training = 0
        file_counter_validation = 0
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.find(self.space_name + "_training") != -1:
                file_counter_training += 1
            elif filename.find(self.space_name + "_validation") != -1:
                file_counter_validation += 1
        self.max_it_train = file_counter_training
        self.max_it_test = file_counter_validation
        self.n_running_avg = self.max_it_test

        self.it_list_train = np.arange(0,self.max_it_train)
        self.it_list_test = np.arange(0,self.max_it_test)

    def get_input(self, flat_input):

        x_OUTDOORTEMPERATURE = torch.zeros((flat_input.shape[0], flat_input.shape[1], 2)).to(DEVICE)
        x_RADIATION = torch.zeros((flat_input.shape[0], flat_input.shape[1], 1)).to(DEVICE)
        x_SPACEHEATER = torch.zeros((flat_input.shape[0], flat_input.shape[1], 3)).to(DEVICE)
        x_VENTILATION = torch.zeros((flat_input.shape[0], flat_input.shape[1], 3)).to(DEVICE)
        
        x_OUTDOORTEMPERATURE[:,:,0] = flat_input[:,:,0] #indoor
        x_OUTDOORTEMPERATURE[:,:,1] = flat_input[:,:,6] #outdoor
        x_RADIATION[:,:,0] = flat_input[:,:,7] #radiation
        x_SPACEHEATER[:,:,0] = flat_input[:,:,0] #indoor
        x_SPACEHEATER[:,:,1] = flat_input[:,:,2] #valve
        x_SPACEHEATER[:,:,2] = flat_input[:,:,3] #supply water temperature
        # x_SPACEHEATER[:,:,3] = flat_input[:,:,2]*flat_input[:,:,3] #energy
        x_VENTILATION[:,:,0] = flat_input[:,:,0] #indoor
        x_VENTILATION[:,:,1] = flat_input[:,:,4] #damper
        x_VENTILATION[:,:,2] = flat_input[:,:,5] #supply air temperature
        # x_VENTILATION[:,:,3] = flat_input[:,:,4]*flat_input[:,:,5] #energy in 
        # x_VENTILATION[:,:,4] = flat_input[:,:,4]*flat_input[:,:,0] #energy out

        x_OUTDOORTEMPERATURE.requires_grad = True
        x_RADIATION.requires_grad = True
        x_SPACEHEATER.requires_grad = True
        x_VENTILATION.requires_grad = True


        input = (x_OUTDOORTEMPERATURE,
                x_RADIATION,
                x_SPACEHEATER,
                x_VENTILATION)

        return input

    def train_batch(self):
        self.model.train()
        #Shuffle data
        idx = np.arange(self.flat_input.shape[0])
        np.random.shuffle(idx)

        self.flat_input = self.flat_input[idx]
        self.output_dT = self.output_dT[idx]


        abs_err_train = 0
        size = self.flat_input.shape[0]
        n_it_sub_batch = int(np.ceil(size/self.n_batch_s))
        t_start = time.time()
        loss_vec = torch.zeros((n_it_sub_batch)).to(DEVICE)
        loss_vec_T = torch.zeros((n_it_sub_batch)).to(DEVICE)
        loss_vec_dT = torch.zeros((n_it_sub_batch)).to(DEVICE)
        update = False
        print("----")
        for idx in range(n_it_sub_batch):
            if idx+1!=n_it_sub_batch or int(np.remainder(size,self.n_batch_s))==0:
                n = self.n_batch_s
            else:
                break
                #n = int(np.remainder(size,n_batch))


            flat_input_batch = self.flat_input[idx*self.n_batch_s:idx*self.n_batch_s+n]
            output_dT_batch = self.output_dT[idx*self.n_batch_s:idx*self.n_batch_s+n]

            input = self.get_input(flat_input_batch)

            # time_start = time.time()
            with torch.backends.cudnn.flags(enabled=False): ####################################################
                output_batch_pred,hidden_state,x = self.model(input, self.hidden_state)
            
            ##### Add temperature MSE to loos function #####
            # dT = rescale(output_batch_pred, self.dT_min, self.dT_max, -1, 1)
            # # T_0 = rescale(output_batch[:,0], self.T_min, self.T_max, -1, 1)
            # T_0 = output_batch[:,0]
            # T_0 = torch.reshape(T_0,(T_0.shape[0],1,T_0.shape[1]))
            # dT_cumsum = torch.cumsum(dT,dim=1)
            # T = T_0 + dT_cumsum
            # T = min_max_norm(T, self.T_min, self.T_max, -1, 1)
            # T_target = min_max_norm(output_batch[:,1:], self.T_min, self.T_max, -1, 1)
            # loss_T = 0*self.loss_fn(T_target,T)
            ################################################


            loss_dT = self.loss_fn(output_batch_pred,output_dT_batch,x,input)
            # loss = loss_T + loss_dT
            loss = loss_dT



            
            loss_vec[idx] = loss.detach()
            # loss_vec_T[idx] = loss_T.detach()
            # loss_vec_dT[idx] = loss_dT.detach()



            # loss_list.append(loss.item())

            self.optimizer.zero_grad()
            loss.backward()

            # for name, W in self.model.named_parameters():
            #     print("### Parameter ###")
            #     print(name)
            #     avg = torch.mean(torch.abs(W.grad))
            #     var = torch.var(W.grad)
            #     print(avg)
            #     print(var)


            self.optimizer.step()
            self.n_step += 1
            # print(idx, time.time()-time_start)

            



            if self.n_step == self.n_step_update and self.n_step <= self.n_step_max:
                update = True
         
                
        if update == True:
            self.learning_rate = self.learning_rate*self.update_factor
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = self.learning_rate
            # self.n_batch_s = int(self.n_batch_s*self.update_factor)
            # self.n_step_diff = int(self.n_step_diff/self.update_factor)
            self.n_step_update = self.n_step_update + self.n_step_diff


        # avg_loss = np.mean(np.array(loss_list))
        
        avg_loss = torch.mean(loss_vec).cpu()
        # avg_loss_T = torch.mean(loss_vec_T).cpu()
        # avg_loss_dT = torch.mean(loss_vec_dT).cpu()

        self.step_train.append(self.n_step)
        self.prec_train.append(avg_loss)
      
        if self.verbose:
            print('Batch number: %d' % (self.batch_idx_train))
            print('Gradient steps: %d' % (self.n_step))
            print('Learning rate: %s' % "{:.10f}".format(self.learning_rate))


            print('Update factor: %d' % (self.update_factor))
            print('Next update: %d' % (self.n_step_update))
            print('Update difference: %d' % (self.n_step_diff))
            print('Max update: %d' % (self.n_step_max))
            print('Batch size: %d' % (self.n_batch_s))

            print('Avg loss: %s' % "{:.10f}".format(avg_loss))


        



    def test_batch(self):

        # with torch.no_grad():
        # self.model.eval()

        os.chdir(self.saved_networks_path)

        rescaled_y_list = []
        rescaled_y_pred_list = []
        r_valve_list = []
        abs_err_test = 0
        size = self.flat_input.shape[0]

        # n_it_sub_batch = int(np.ceil(size/self.n_batch_s))
        n_it_sub_batch = int(np.ceil(size/self.file_batch))
        # loss_list = []
        loss_vec = torch.zeros((n_it_sub_batch)).to(DEVICE)
        for idx in range(n_it_sub_batch):
            if idx+1!=n_it_sub_batch or int(np.remainder(size,self.n_batch_s))==0:
                n = self.n_batch_s
            else:
                break


            flat_input_batch = self.flat_input[idx*self.n_batch_s:idx*self.n_batch_s+n]
            output_dT_batch = self.output_dT[idx*self.n_batch_s:idx*self.n_batch_s+n]


            input = self.get_input(flat_input_batch)
            output_batch_pred,hidden_state,x = self.model(input, self.hidden_state)


            ##### Add temperature MSE to loos function #####
            # dT = rescale(output_batch_pred, self.dT_min, self.dT_max, -1, 1)
            # # T_0 = rescale(output_batch[:,0], self.T_min, self.T_max, -1, 1)
            # T_0 = output_batch[:,0]
            # T_0 = torch.reshape(T_0,(T_0.shape[0],1,T_0.shape[1]))
            # dT_cumsum = torch.cumsum(dT,dim=1)
            # T = T_0 + dT_cumsum
            # T = min_max_norm(T, self.T_min, self.T_max, -1, 1)
            # T_target = min_max_norm(output_batch[:,1:], self.T_min, self.T_max, -1, 1)
            # loss_T = 0*self.loss_fn(T_target,T)
            ################################################

            loss_dT = self.loss_fn(output_batch_pred,output_dT_batch,x,input)
            # loss = loss_T + loss_dT
            loss = loss_dT


            # loss = self.loss_fn(output_batch_pred,output_batch)
            loss_vec[idx] = loss.detach()
            # loss_list.append(loss.item())

            rescaled_y = rescale(output_dT_batch.cpu(),self.dT_min,self.dT_max,-1,1)
            rescaled_y_pred = rescale(output_batch_pred.cpu(),self.dT_min,self.dT_max,-1,1)

            rescaled_y_list.extend(np.array(rescaled_y))
            rescaled_y_pred_list.extend(np.array(rescaled_y_pred.detach()))
            r_valve_list.extend(np.array(flat_input_batch[:,:,1].detach().cpu()))

        # avg_loss = np.mean(np.array(loss_list))
        avg_loss = torch.mean(loss_vec).cpu()

        if self.verbose:
            # print('Running average: %s' % "{:.10f}".format(running_test_avg))
            print('Avg loss: %s' % "{:.10f}".format(avg_loss))
        

        self.rescaled_y_list = np.array(rescaled_y_list)
        self.rescaled_y_pred_list = np.array(rescaled_y_pred_list)
        self.r_valve_list = np.array(r_valve_list)

        self.step_test.append(self.n_step)
        self.prec_test.append(avg_loss)


        #Saving
        np.save(self.step_train_filename_save + ".npy",np.array(self.step_train))
        np.save(self.prec_train_filename_save + ".npy",np.array(self.prec_train))

        np.save(self.step_test_filename_save + ".npy",np.array(self.step_test))
        np.save(self.prec_test_filename_save + ".npy",np.array(self.prec_test))

        torch.save(self.model.state_dict(),self.network_filename_save)
        torch.save(self.optimizer.state_dict(),self.optimizer_filename_save)

        

            

    def test_all(self):
        for i in range(self.max_it_test):
            self.load_batch_into_GPU()
            self.test_batch()
            self.batch_idx_test += 1
        self.model.eval()

        if self.plot:
            # running_test_avg = np.mean(np.array(self.prec_test[-self.n_running_avg:]))
            running_test_avg = torch.mean(torch.Tensor(self.prec_test[-self.n_running_avg:]))
            np_step_test = np.array(self.step_test, dtype=np.int)
            np_prec_test = np.array(self.prec_test)
            unique_step_test = np.unique(self.step_test)
            unique_prec_test = [np.mean(np_prec_test[idx==np_step_test]) for idx in unique_step_test]
            try:
                self.const_line.remove()
            except:
                pass

           
            if self.plot:
                # Plotting
                self.ax_loss.plot(self.step_train,self.prec_train, color="blue")
                self.ax_loss.plot(unique_step_test,unique_prec_test, color="black")
                self.const_line = self.ax_loss.axhline(y=running_test_avg, color='r', linestyle='-')
                self.ax_loss.set_yscale('log')
                # display.display(plt.gcf())

                # fig_profile.clear(True) 


                self.ax[0].clear()
                var_vec = np.var(self.rescaled_y_list[:,:,0], axis=1)
                max_var_idx = np.random.randint(self.rescaled_y_list[:,:,0].shape[0])#np.argmax(var_vec)
                
                self.ax[0].plot(self.rescaled_y_list[max_var_idx,:,0], color="blue",label="Truth")
                self.ax[0].plot(self.rescaled_y_pred_list[max_var_idx,:,0], color="green",label="Prediction")
                self.ax[0].plot(self.r_valve_list[max_var_idx,:], color="red",label="valve")
                # ax_i.set_ylim([15, 28])
                self.ax[0].legend()


                self.ax[1].clear()
                self.ax[1].plot(self.output.cpu().detach().numpy()[max_var_idx,:-1,0]) #
                # self.ax[1].set_ylim([15, 28])

                display.display(plt.gcf())
                # display.clear_output(wait=True)
                time.sleep(0.1)

    def serialize_model(self):
        do_break = False
        # os.chdir(S_path)

        self.model.cpu()


        os.chdir(self.saved_serialized_networks_path)
        filename = self.network_filename_save + "_step" + str(self.n_step) + ".pt"
        torch.save((self.model.kwargs, self.model.state_dict()), filename)

        
        self.running_validation_loss.append(np.mean(np.array(self.prec_test[-self.n_running_avg:])))
        np.save(self.running_validation_loss_filename_save + ".npy",np.array(self.running_validation_loss))

        self.running_validation_loss_model_name.append(self.network_filename_save + "_step" + str(self.n_step) + ".pt")
        np.save(self.running_validation_loss_model_name_filename_save + ".npy",np.array(self.running_validation_loss_model_name))

        if self.verbose:
            print("Saved serialized module")
        
        best_loss_diff = (len(self.running_validation_loss)-np.nanargmin(np.array(self.running_validation_loss))-1)*self.n_save_traced_module
        if best_loss_diff>=self.best_loss_diff_max:
            print("No improvement for the last " + str(self.best_loss_diff_max) + " iterations: Stopping...")
            do_break = True

        self.extract_model = False

        self.model.train()
        self.model.to(DEVICE)

        self.sort_directory()

        return do_break

    def train(self, verbose=False):

        self.verbose = verbose
        
        if self.plot:
            rows = 3
            fig = plt.figure(figsize=(40,10))
            grid = plt.GridSpec(rows, 1, hspace=0.2, wspace=0.2)
            self.ax_loss = fig.add_subplot(grid[0, 0:1]) #0:2


            self.ax = []
            for i in range(1,rows,1):
                for j in range(1):
                    self.ax.append(fig.add_subplot(grid[i, j]))#, xticklabels=[])#, sharey=main_ax)

            for ax_i in self.ax:
                ax_i.set_ylim([20, 23])
        self.first_test = True

        
        while True:   

            

            if self.verbose:
                print('--------------------------') 


            self.load_batch_into_GPU()

            if self.do_test == True:
                self.test_all()
                # self.batch_idx_test += 1
                self.do_test = False

            self.train_batch()
            self.batch_idx_train += 1

            if self.extract_model==True:
                do_break = self.serialize_model()
                if do_break:
                    break

            if self.n_step % self.n_test == 0:
                self.do_test = True

            if self.n_step % self.n_save_traced_module == 0:
                self.extract_model = True

            
            best_loss_diff = (len(self.running_validation_loss)-np.nanargmin(np.array(self.running_validation_loss))-1)*self.n_save_traced_module
            running_test_avg = torch.mean(torch.Tensor(self.prec_test[-self.n_running_avg:]))
            add_args = [self.n_step, running_test_avg.item(), best_loss_diff, np.nanmin(np.array(self.running_validation_loss))]
            progressbar(self.n_step,self.n_step_start,self.max_it_stop, add_args = add_args)
                

            
            self.batch_train_counter += 1
        #    
            # print('--------------------------')
            if self.n_step >= self.max_it_stop:
                break

            if time.time()-start_time>time_limit:
                break

    def display(self,str_input):

        fill_str = "#"
        n = 100
        n_str_input = len(str_input)
        n_fill_left = math.ceil((n-n_str_input)/2) - 1
        n_fill_right = n-n_fill_left-n_str_input - 2
        n_rows = 3
        i_row = 1 #Index of row placement of input string

        str_output = ""
        for row in range(n_rows):
            if row == i_row:
                for j in range(n_fill_left):
                    str_output += fill_str
                str_output += " "
                str_output += str_input
                str_output += " "
                for j in range(n_fill_right):
                    str_output += fill_str
                str_output += "\n"
            else:
                for j in range(n):
                    str_output += fill_str
                str_output += "\n"

        print(str_output)



    def sort_directory(self):
        os.chdir(self.saved_serialized_networks_path)

        running_validation_loss = np.array(self.running_validation_loss)
        idx = np.argmin(running_validation_loss)
        running_validation_loss_model_name_sorted = self.running_validation_loss_model_name[:]
        running_validation_loss_model_name_sorted.pop(idx)

        for filename in running_validation_loss_model_name_sorted:
            try:
                os.remove(filename)
            except:
                pass

        self.running_validation_loss = [self.running_validation_loss[idx]]
        self.running_validation_loss_model_name = [self.running_validation_loss_model_name[idx]]
        
        np.save(self.running_validation_loss_filename_save + ".npy",np.array(self.running_validation_loss))
        np.save(self.running_validation_loss_model_name_filename_save + ".npy",np.array(self.running_validation_loss_model_name))

    def one_time_delete(self):
        os.chdir(self.saved_serialized_networks_path)

        running_validation_loss = np.array(self.running_validation_loss)
        idx = np.argmin(running_validation_loss)

        print("----")
        print(self.running_validation_loss_model_name)
        print(self.running_validation_loss)

        if running_validation_loss.size!=1:
            

            self.running_validation_loss = [self.running_validation_loss[idx]]
            self.running_validation_loss_model_name = [self.running_validation_loss_model_name[idx]]

            np.save(self.running_validation_loss_filename_save + ".npy",np.array(self.running_validation_loss))
            np.save(self.running_validation_loss_model_name_filename_save + ".npy",np.array(self.running_validation_loss_model_name))

        print("----")
        print(self.running_validation_loss_model_name)
        print(self.running_validation_loss)

def load_dataset_into_disk(data_paths):
    file_batch = 2**9
    (training_data_folder_name,
    drive_zipped_training_data_path,
    drive_unzipped_training_data_path,
    vm_drive_unzip_data_path,
    vm_drive_training_data_path) = data_paths
    
    print(drive_zipped_training_data_path)
    print(vm_drive_unzip_data_path)
    
    if os.path.isdir(training_data_folder_name):
        print("VM training directory already exists: Proceeding...")
    else:
        print("Unzipping training directory...")
        command_str = "unzip \"%s\" -d \"%s\" " % (drive_zipped_training_data_path, vm_drive_unzip_data_path)
        os.system(command_str)
    
    # os.chdir(vm_drive_training_data_path)
    # filename =  "saved_space_list" + ".npz"
    # loaded = np.load(filename)
    # saved_space_list = loaded[loaded.files[0]]
    
    
    print("Loaded...")
    # new_saved_space_list = []

    # print("Validating dataset...")

    # os.chdir(vm_drive_training_data_path)
    # for i,space_name in enumerate(saved_space_list):
        
    #     filename1 = space_name + "_validation_batch_%d.npz" % file_batch
    #     filename2 = space_name + "_test_batch_%d.npz" % file_batch
    #     if os.path.isfile(filename1) and os.path.isfile(filename2):
    #         load = True
    #     else:
    #         print("Can't load \"" + space_name + "\"")
    #         load = False

    #     if load == False:
    #         i = 0
    #         while True:
    #             name_idx = file_batch*i + file_batch
    #             filename = space_name + "_" + "training" + "_batch_%d.npz" % (name_idx)
    #             try:
    #                 os.remove(filename)
    #             except Exception as inst:
    #                 break
    #             i += 1

    #     else:
    #         new_saved_space_list.append(space_name)

    # os.chdir(vm_drive_unzip_data_path)
    # filename =  "saved_space_list" + ".npz"
    # np.savez_compressed(filename, np.array(new_saved_space_list))

    # return new_saved_space_list


def progressbar(current,start,stop, add_args=None):

    

    total_time = stop-start
    relative_time = current-start

    n_ticks_total = 40
    n_ticks_current = math.ceil(relative_time/total_time*n_ticks_total)
    percent_done = math.ceil(relative_time/total_time*100)

    progress_str = '|'
    for i in range(n_ticks_total):
        if n_ticks_current > i:
            progress_str += '#'
        else:
            progress_str += '-'
    
    if add_args:
        progress_str += '| ' + str(percent_done) + '% '
        for arg in add_args:
            progress_str += "-- " + str(arg) + " "
    else:
        progress_str += '| ' + str(percent_done) + '% '


    sys.stdout.write('\r\x1b[K' + progress_str)
    sys.stdout.flush()

# !nvidia-smi
# drive.mount('/content/drive')

start_time = time.time()
time_limit = 1e+10





drive.mount('/content/drive')
training_data_folder_name = "space_model_batches"
drive_zipped_training_data_path = "/content/drive/My Drive/Google collab/OU44_zipped" + "/" + training_data_folder_name + ".zip"
drive_unzipped_training_data_path = "/content/drive/My Drive/Google collab/OU44_LSTM_data_2048batch_-11norm_10min_144seq_filtered_test_relu/"
vm_drive_unzip_data_path = "/content"
vm_drive_training_data_path = vm_drive_unzip_data_path + "/" + training_data_folder_name

data_paths = (training_data_folder_name,
            drive_zipped_training_data_path,
            drive_unzipped_training_data_path,
            vm_drive_unzip_data_path,
            vm_drive_training_data_path)

saved_space_list = load_dataset_into_disk(data_paths)

# trainer.load_dataset_into_disk()
# trainer.load_batch_into_GPU()
# trainer.visualize_outputs()


# trainer.load_batch_into_GPU()
# trainer.get_input_gradients()






######################################################
# for i,space_name in enumerate(saved_space_list):
#     # clear_output(wait=True)
#     train = False
#     try:
#         trainer = Trainer(space_name, data_paths, load=True)
#         running_validation_loss = np.array(trainer.running_validation_loss)

#         if running_validation_loss.size==1:
#             print("Space \"" + space_name + "\" exists and has size 1")
#             # print(trainer.running_validation_loss_model_name)

#             best_idx = int(trainer.running_validation_loss_model_name[0].split("_step")[1].split(".pt")[0])

            
#             best_loss_diff = trainer.n_step-best_idx
#             if best_loss_diff>=trainer.best_loss_diff_max:
#                 print("No improvement for the last " + str(trainer.best_loss_diff_max) + " iterations: Skipping...")
#             else:
#                 train = True

#         else:
#             print("Space \"" + space_name + "\" exists but has size " + str(running_validation_loss.size))
#             trainer = Trainer(space_name, data_paths, load=True)
#             train = True

#     except Exception as inst:
#         # print(inst)
#         print("Can't load \"" + space_name + "\"")
#         trainer = Trainer(space_name, data_paths, load=False)
#         train = True

#     if train == True:
#         print("Space number " + str(i))
#         trainer.load_min_max_scale_values()
#         trainer.scan_directory()
#         trainer.train(verbose=False)
#         trainer.sort_directory()
##################################################################
# Ø20-601b-2
# Ø22-511-2
space_name = "OE20-601b-2"
# space_name = "OE22-511-2"
trainer = Trainer(space_name, data_paths, load=False, plot=False)
trainer.scan_directory()
trainer.train()
trainer.sort_directory()



# %mprun -f trainer.load_dataset_into_RAM trainer.load_dataset_into_RAM()
# %lprun -f trainer.train_batch -f trainer.test_batch -f trainer.train -f trainer.load_batch_into_GPU trainer.train()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Google collab/OU44_zipped/space_model_batches.zip
/content
Unzipping training directory...
Loaded...
INPUT
8
TOTAL NUMBER OF PARAMETERS IN MODEL: 2295
----
|#---------------------------------------| 1% -- 8 -- 0.03811035305261612 -- 0 -- 0.038110357 ----
|#---------------------------------------| 1% -- 16 -- 0.03811035305261612 -- 0 -- 0.038110357 ----
|#---------------------------------------| 1% -- 24 -- 0.022813213989138603 -- 0 -- 0.022813214 ----
|#---------------------------------------| 1% -- 32 -- 0.022813213989138603 -- 0 -- 0.022813214 ----
|#---------------------------------------| 1% -- 40 -- 0.02278056927025318 -- 0 -- 0.02278057 ----
|#---------------------------------------| 1% -- 48 -- 0.02278056927025318 -- 0 -- 0.02278057 ----
|#---------------------------------------| 1% -- 56 -- 0.022716457024216652 -- 0 -- 0.022716

KeyboardInterrupt: ignored

In [ ]:
import sys
import math

def progressbar(current,start,stop, add_args=None):

    

    total_time = stop-start
    relative_time = current-start

    n_ticks_total = 40
    n_ticks_current = math.ceil(relative_time/total_time*n_ticks_total)
    percent_done = math.ceil(relative_time/total_time*100)

    progress_str = '|'
    for i in range(n_ticks_total):
        if n_ticks_current > i:
            progress_str += '#'
        else:
            progress_str += '-'
    
    if add_args:
        progress_str += '| ' + str(percent_done) + '% '
        for arg in add_args:
            progress_str += "-- " + str(arg)
    else:
        progress_str += '| ' + str(percent_done) + '% '


    sys.stdout.write('\r\x1b[K' + progress_str)
    sys.stdout.flush()


for i in range(10000):
    progressbar(i,0,10000)



|########################################| 100% 

In [ ]:
from main import Trainer
import psutil

!pip install memory_profiler
# !pip install psutil
%load_ext memory_profiler

trainer = Trainer()
%mprun -f trainer.load_dataset_into_RAM trainer.load_dataset_into_RAM()


  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31285 sha256=8beee7403bc291796459a59f64a8832243f92548bd659677491ff3f7f4322478
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler
Mounted at /content/drive



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)



TOTAL NUMBER OF PARAMETERS IN MODEL: 2643209
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0
----
4
905969664
3623878656
1



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



----
4
905969664
3623878656



In [ ]:
import sys
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
